In [24]:
## Load required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

## xgboost model
import pickle
import xgboost as xgb
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

# Dataset
from aif360.datasets import BinaryLabelDataset

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler

# Bias mitigation techniques
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.preprocessing import DisparateImpactRemover
from aif360.algorithms.preprocessing import LFR
from aif360.algorithms.preprocessing import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

# Odds equalizing post-processing algorithm
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import tensorflow session
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [64]:
from collections import defaultdict

def test(dataset, model, thresh_arr, unprivileged_groups, privileged_groups):
    try:
        # sklearn classifier
        y_val_pred_prob = model.predict_proba(dataset.features)
        pos_ind = 1
    except AttributeError:
        # aif360 inprocessing algorithm
        y_val_pred_prob = model.predict(dataset).scores
        pos_ind = 0
    
    metric_arrs = defaultdict(list)
    for thresh in thresh_arr:
        y_val_pred = (y_val_pred_prob[:, pos_ind] > thresh).astype(np.float64)

        dataset_pred = dataset.copy()
        dataset_pred.labels = y_val_pred
        metric = ClassificationMetric(
                dataset, dataset_pred,
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)

        ## Balanced Accuracy (TPR + TNR)/2
        metric_arrs['bal_acc'].append((metric.true_positive_rate()
                                     + metric.true_negative_rate()) / 2)
        ## Average Odds Difference (average difference between TPR and TNR of groups)
        metric_arrs['avg_odds_diff'].append(metric.average_odds_difference())
        ## Whether ratio of favorable outcome is consistent
        metric_arrs['disp_imp'].append(metric.disparate_impact())
        ## Difference in probability of receiving favorable outcome
        metric_arrs['stat_par_diff'].append(metric.statistical_parity_difference())
        ## Whether individual have equal chances of receiving positive outcome when they should
        metric_arrs['eq_opp_diff'].append(metric.equal_opportunity_difference())
        ## Measure inequality of model errors across group
        metric_arrs['theil_ind'].append(metric.theil_index())
    
    return metric_arrs

In [109]:
def plot_results(x, x_name, y_left, y_left_name, y_right, y_right_name, title = None):
    fig, ax1 = plt.subplots(figsize=(10,7))
    ax1.plot(x, y_left)
    ax1.set_xlabel(x_name, fontsize=16, fontweight='bold')
    ax1.set_ylabel(y_left_name, color='b', fontsize=16, fontweight='bold')
    ax1.xaxis.set_tick_params(labelsize=14)
    ax1.yaxis.set_tick_params(labelsize=14)
    ax1.set_ylim(0.5, 0.8)

    ax2 = ax1.twinx()
    ax2.plot(x, y_right, color='r')
    ax2.set_ylabel(y_right_name, color='r', fontsize=16, fontweight='bold')
    if 'DI' in y_right_name:
        ax2.set_ylim(0., 0.7)
    else:
        ax2.set_ylim(-0.25, 0.1)

    best_ind = np.argmax(y_left)
    ax2.axvline(np.array(x)[best_ind], color='k', linestyle=':')
    ax2.yaxis.set_tick_params(labelsize=14)
    ax2.grid(True)
    if title is not None:
        ax1.set_title(title)

## Comparison of the Four Pre-Processing Methods in AIF360

This report is for the Milestone paper, here we compare as many methods as we can from AIF360 to get a comprehensive evaluation of how well the model performs on the Taiwanese dataset.

In [33]:
## Load in the dataset
data = pd.read_csv(
    "../../data/default_clean_v1.csv",
    index_col=0,
    dtype={'default':"category",
           "SEX":"category",
           "EDUCATION":"category",
           "MARRIAGE":"category"}
)

# Convert the default, sex, education, marriage variables to categorical
data["default"] = data["default"].cat.codes
data["SEX"] = data["SEX"].cat.codes

# Drop the extra default column
data.drop(columns = ["default payment next month"], inplace = True)

categorical_cols = ["EDUCATION", "MARRIAGE"]
data_dum = pd.get_dummies(data, columns=categorical_cols)


# Initiate the binary label dataset - AIF360 convention
binary_data = BinaryLabelDataset(
    favorable_label=0,
    unfavorable_label=1,
    df = data_dum,
    label_names = ["default"],
    protected_attribute_names = ["SEX"],
)

# Split data into training and testing sets
data_train, data_test = binary_data.split([0.7], shuffle=True)

# Set unpriviledged groups and priviledged groups
privileged_groups = [{'SEX': 0}]
unprivileged_groups = [{'SEX': 1}]

### Make base model to assess Disparate Impact

In [65]:
X_train = data_train.convert_to_dataframe()[0].drop(columns = ["default"])
y_train = data_train.convert_to_dataframe()[0]["default"]
X_test = data_test.convert_to_dataframe()[0].drop(columns = ["default"])
y_test = data_test.convert_to_dataframe()[0]["default"]

## Build model 
xgb_model = xgb.XGBClassifier(random_state = 2023)
param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}

## Conduct Random Search
random_search = RandomizedSearchCV(xgb_model, 
                                   param_distributions=param_dist, 
                                   n_iter = 10, 
                                   cv = 5, 
                                   scoring="roc_auc",
                                   random_state=2023)
random_search.fit(X_train, y_train)
best_xgb_original = random_search.best_estimator_
thresh_arr = np.linspace(0.01, 0.5, 50)

## Generate Metrics
val_metrics = test(dataset=data_test,
                   model=best_xgb_original,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups,
                   privileged_groups=privileged_groups)

lr_orig_best_ind = np.argmax(val_metrics['bal_acc'])
original_metrics = {
    "name":"Vanilla XGBoost",
    "balanced accuracy":val_metrics["bal_acc"][lr_orig_best_ind],
    "disparate impact":val_metrics["disp_imp"][lr_orig_best_ind]
}
print(original_metrics)

/Users/genesisqu/miniconda3/lib/python3.11/site-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: invalid value encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


{'name': 'Vanilla XGBoost', 'balanced accuracy': 0.7090393182610983, 'disparate impact': 0.92911088770505}


### Use Reweighting Scheme


In [48]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
RW.fit(data_train)
data_train_reweighed = RW.transform(data_train)

X_train = data_train_reweighed.convert_to_dataframe()[0].drop(columns = ["default"])
y_train = data_train_reweighed.convert_to_dataframe()[0]["default"]
X_test = data_train_reweighed.convert_to_dataframe()[0].drop(columns = ["default"])
y_test = data_train_reweighed.convert_to_dataframe()[0]["default"]

## Build model 
xgb_model = xgb.XGBClassifier()
param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}

## Conduct Random Search
fit_param = {"sample_weight":data_train_reweighed.instance_weights}
random_search = RandomizedSearchCV(xgb_model, 
                                   param_distributions=param_dist, 
                                   n_iter = 10, 
                                   cv = 5, 
                                   scoring="roc_auc", 
                                   random_state=2023)
random_search.fit(X_train, y_train, **fit_param)
best_xgb_reweighed = random_search.best_estimator_
thresh_arr = np.linspace(0.01, 0.5, 50)

## Generate Metrics
val_metrics_reweigh = test(dataset=data_test,
                   #### Change Model ####
                   model=best_xgb_reweighed,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups,
                   privileged_groups=privileged_groups)
lr_reweight_best_ind = np.argmax(val_metrics_reweigh['bal_acc'])
#### Change Metric name ####
reweight_metrics = {
    #### Change Name ####
    "name":"XGBoost after Reweighting",
    "balanced accuracy":val_metrics_reweigh["bal_acc"][lr_reweight_best_ind],
    "disparate impact":val_metrics_reweigh["disp_imp"][lr_reweight_best_ind]
}
print(reweight_metrics)

/Users/genesisqu/miniconda3/lib/python3.11/site-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: invalid value encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


{'name': 'XGBoost after Reweighting', 'balanced accuracy': 0.70948416804575, 'disparate impact': 0.9385394511281405}


### Preprocessing: LFR

Use Learned Feature Representation to create new fairer feature independent of protected variables.

In [66]:
lfr = LFR(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
lfr.fit(data_train)
data_train_lfr = lfr.transform(data_train)

X_train = data_train_lfr.convert_to_dataframe()[0].drop(columns = ["default"])
y_train = data_train_lfr.convert_to_dataframe()[0]["default"]
X_test = data_train_lfr.convert_to_dataframe()[0].drop(columns = ["default"])
y_test = data_train_lfr.convert_to_dataframe()[0]["default"]

## Build model 
xgb_model = xgb.XGBClassifier(random_state = 2023)
param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}

## Conduct Random Search
random_search = RandomizedSearchCV(xgb_model, 
                                   param_distributions=param_dist, 
                                   n_iter = 10, 
                                   cv = 5, 
                                   scoring="roc_auc")
random_search.fit(X_train, y_train)
best_xgb_lfr = random_search.best_estimator_
thresh_arr = np.linspace(0.01, 0.5, 50)

## Generate Metrics
val_metrics_lfr = test(dataset=data_test,
                   #### Change Model ####
                   model=best_xgb_lfr,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups,
                   privileged_groups=privileged_groups)
lr_lfr_best_ind = np.argmax(val_metrics_lfr['bal_acc'])
#### Change Metric name ####
lfr_metrics = {
    #### Change Name ####
    "name":"XGBoost with LFR",
    "balanced accuracy":val_metrics_lfr["bal_acc"][lr_lfr_best_ind],
    "disparate impact":val_metrics_lfr["disp_imp"][lr_lfr_best_ind]
}
print(lfr_metrics)

{'name': 'XGBoost with LFR', 'balanced accuracy': 0.4965956512189765, 'disparate impact': 1.0497296357381716}


### Using Disparate Impact Remover

In [67]:
di = DisparateImpactRemover(repair_level=1, sensitive_attribute="SEX")
data_train_di = di.fit_transform(data_train)

X_train = data_train_di.convert_to_dataframe()[0].drop(columns = ["default"])
y_train = data_train_di.convert_to_dataframe()[0]["default"]
X_test = data_train_di.convert_to_dataframe()[0].drop(columns = ["default"])
y_test = data_train_di.convert_to_dataframe()[0]["default"]

## Build model 
xgb_model = xgb.XGBClassifier(random_state = 2023)
param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}

## Conduct Random Search
random_search = RandomizedSearchCV(xgb_model, 
                                   param_distributions=param_dist, 
                                   n_iter = 10, 
                                   cv = 5, 
                                   scoring="roc_auc")
random_search.fit(X_train, y_train)
best_xgb_di = random_search.best_estimator_
thresh_arr = np.linspace(0.01, 0.5, 50)

## Generate Metrics
val_metrics = test(dataset=data_test,
                   #### Change Model ####
                   model=best_xgb_di,
                   thresh_arr=thresh_arr,
                   unprivileged_groups=unprivileged_groups,
                   privileged_groups=privileged_groups)
lr_di_best_ind = np.argmax(val_metrics['bal_acc'])
#### Change Metric name ####
di_metrics = {
    #### Change Name ####
    "name":"XGBoost with Disparate Impact Remover",
    ### Change the indices here ####
    "balanced accuracy":val_metrics["bal_acc"][lr_di_best_ind],
    "disparate impact":val_metrics["disp_imp"][lr_di_best_ind]
}
#### Change the print statement ####
print(di_metrics)

/Users/genesisqu/miniconda3/lib/python3.11/site-packages/aif360/metrics/dataset_metric.py:82: RuntimeWarning: invalid value encountered in scalar divide
  return metric_fun(privileged=False) / metric_fun(privileged=True)


{'name': 'XGBoost with Disparate Impact Remover', 'balanced accuracy': 0.7112519687906311, 'disparate impact': 0.9481187766809394}


### Try Post-Processing Methods 1: Calibrated Equality of Odds

In [76]:
### Learn parameters to equalize odds and apply to create a new dataset
dataset_test_cpp = data_test.copy(deepcopy=True)
cost_constraint = "fnr"
# Make predictions using original xgboost model
y_pred_original_prob = best_xgb_original.predict_proba(data_test.features)[:,0]
dataset_test_cpp.scores = y_pred_original_prob.reshape(-1,1)
### Get labels
best_threshold = thresh_arr[lr_orig_best_ind]
y_test_pred_original = np.zeros_like(dataset_test_cpp.labels)
y_test_pred_original[y_pred_original_prob >= best_threshold] = dataset_test_cpp.favorable_label
y_test_pred_original[~(y_pred_original_prob >= best_threshold)] = dataset_test_cpp.unfavorable_label
dataset_test_cpp.labels = y_test_pred_original


cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint=cost_constraint,
                                     seed=2023)

# Fit and transform using calibrated equality of odds
cpp = cpp.fit(data_test, dataset_test_cpp)
data_test_cpp_new = cpp.predict(dataset_test_cpp)
## Get classification accuracy and fairness metric
cm_transf_valid = ClassificationMetric(data_test, data_test_cpp_new,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
cp_tp = cm_transf_valid.true_positive_rate()
cp_tn = cm_transf_valid.true_negative_rate()
cp_fp = cm_transf_valid.false_positive_rate()
cp_fn = cm_transf_valid.false_negative_rate()
cp_balanced_acc = (cp_tp + cp_tn)/2
cp_metrics = {
    #### Change Name ####
    "name":"Calibrated Equality of Odds",
    ### Change the indices here ####
    "balanced accuracy":cp_balanced_acc,
    "disparate impact":cm_transf_valid.disparate_impact()
}
print(cp_metrics)

{'name': 'Calibrated Equality of Odds', 'balanced accuracy': 0.6183875942406194, 'disparate impact': 0.9169900305285925}


### Reject Options Classification

In [74]:
# roc = RejectOptionClassification(
#     unprivileged_groups,
#     privileged_groups
# )
# data_test_roc = roc.fit_predict(data_test, dataset_test_cpp)

# ## Get classification accuracy and fairness metric
# cm_roc = ClassificationMetric(data_test, 
#                               data_test_roc,
#                              unprivileged_groups=unprivileged_groups,
#                              privileged_groups=privileged_groups)
roc_tp = cm_roc.true_positive_rate()
roc_tn = cm_roc.true_negative_rate()

roc_balanced_acc = (roc_tp + roc_tn)/2
roc_metrics = {
    #### Change Name ####
    "name":"Reject Option Classification",
    ### Change the indices here ####
    "balanced accuracy":roc_balanced_acc,
    "disparate impact":cm_roc.disparate_impact()
}
print(roc_metrics)

{'name': 'Reject Option Classification', 'balanced accuracy': 0.708576116587641, 'disparate impact': 0.9405863176931226}


## Plots

In [16]:
# Define plotting parameters
%matplotlib inline
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

In [104]:
dicts = [original_metrics, reweight_metrics, lfr_metrics, di_metrics, cp_metrics, roc_metrics]
super_dict = {}
for d in dicts:
    for k, v in d.items():  # d.items() in Python 3+
        super_dict.setdefault(k, []).append(v)

In [70]:
import json
# # Serializing json
# json_object = json.dumps(super_dict, indent=4)
 
# # Writing to sample.json
# with open("aif_compare.json", "w") as outfile:
#     outfile.write(json_object)

# Opening JSON file
with open('aif_compare.json', 'r') as openfile:
    # Reading from json file
    aif_compare = json.load(openfile)

In [71]:
fig, axes = plt.subplots(1,2,figsize = (10,5), sharey = True)
axes[0].barh(np.arange(len(aif_compare["name"])), np.array(aif_compare["balanced accuracy"]) * 100)
axes[0].set_yticks(np.arange(len(aif_compare["name"])), labels=aif_compare["name"])
axes[0].axvline(x = aif_compare["balanced accuracy"][0] * 100, linestyle = "dashed", c = "red")
axes[0].xaxis.set_major_formatter(mtick.PercentFormatter())
axes[0].set_title("Balanced Accuracy")
axes[1].barh(np.arange(len(aif_compare["name"])), np.abs(1 - np.array(aif_compare["disparate impact"])) * 100)
axes[1].set_yticks(np.arange(len(aif_compare["name"])), labels=aif_compare["name"])
axes[1].axvline(x = (1 - aif_compare["disparate impact"][0]) * 100, linestyle = "dashed", c = "red", label = "Vanilla Baseline")
axes[1].set_title("|1 - Disparate Impact|")
axes[1].xaxis.set_major_formatter(mtick.PercentFormatter())
axes[1].legend()
fig.suptitle("AIF360 Fairness Mitigation Metrics")
plt.show()

/var/folders/0d/23x0hjl172sgh7hhvyngkf9m0000gn/T/ipykernel_30999/3597319341.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
